# <font color=darkblue> Machine Learning model deployment with Flask framework on Railway.app</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with flask framework on Railway.app.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [9]:
## Import the libraries 

import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline 

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import r2_score, accuracy_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

import pickle

### 2. Load the dataset

In [10]:
# Loading the dataset

df = pd.read_csv('car data.csv')
df.head(5)

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


### 3. Check the shape and basic information of the dataset.

In [11]:
# Checking shape of the dataset

df.shape

(301, 9)

In [12]:
# Checking basic information about the dataset

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


- There are 301 rows & 9 fields in the dataset.
- No null values present in any of the fields.

### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [13]:
df[df.duplicated()]

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
17,ertiga,2016,7.75,10.79,43000,Diesel,Dealer,Manual,0
93,fortuner,2015,23.00,30.61,40000,Diesel,Dealer,Automatic,0


- There are 2 duplicated rows in the dataset

In [14]:
# Removing the duplicated rows

df = df.drop_duplicates()

df.shape

(299, 9)

- After removal of 2 duplicate rows no of rows reduces from 301 to 299

### 5. Drop the columns which you think redundant for the analysis.

In [15]:
# Dropping the redundant columns

df = df.drop(columns=['Car_Name'], axis=1)
df.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


- We drop the 'Car_Name' column which is redundant as there can be many variation under the same Car Name

### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [16]:
import datetime

# Geting age of the car
curr_year = datetime.date.today().year
df['age_of_the_car'] = curr_year - df['Year']

# Dropping 'Year' column after we get the age of the car
df = df.drop(columns=['Year'])

df.head()

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,3.35,5.59,27000,Petrol,Dealer,Manual,0,10
1,4.75,9.54,43000,Diesel,Dealer,Manual,0,11
2,7.25,9.85,6900,Petrol,Dealer,Manual,0,7
3,2.85,4.15,5200,Petrol,Dealer,Manual,0,13
4,4.60,6.87,42450,Diesel,Dealer,Manual,0,10


- A new column 'age_of_the_car' is extarcted from the Year column by the feature engineering methodology
- The 'Year' column is dropped after extracting the age as it is now redundant.

### 7. Encode the categorical columns

In [21]:
# Getting the ctegorical columns which needs encoding

categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
categorical_columns


['Fuel_Type', 'Seller_Type', 'Transmission']

- We find presence of Three categorcal columns in the dataset - 'Fuel_Type', 'Seller_Type', 'Transmission'

In [22]:
# Getting the unique values in the 'Fuel_Type' column
df['Fuel_Type'].unique()


array(['Petrol', 'Diesel', 'CNG'], dtype=object)

In [23]:
# We employ manual encoding of the 'Fuel_Type' column

df['Fuel_Type'] = df['Fuel_Type'].replace({'Petrol':0,'Diesel':1,'CNG':2})
df.head()

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,3.35,5.59,27000,0,Dealer,Manual,0,10
1,4.75,9.54,43000,1,Dealer,Manual,0,11
2,7.25,9.85,6900,0,Dealer,Manual,0,7
3,2.85,4.15,5200,0,Dealer,Manual,0,13
4,4.60,6.87,42450,1,Dealer,Manual,0,10


In [57]:
df['Seller_Type'].unique()

array(['Dealer', 'Individual'], dtype=object)

In [58]:
df['Transmission'].unique()

array(['Manual', 'Automatic'], dtype=object)

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,3.35,5.59,27000,0,Dealer,Manual,0,10
1,4.75,9.54,43000,1,Dealer,Manual,0,11
2,7.25,9.85,6900,0,Dealer,Manual,0,7
3,2.85,4.15,5200,0,Dealer,Manual,0,13
4,4.60,6.87,42450,1,Dealer,Manual,0,10


In [60]:
df['Seller_Type'] = df['Seller_Type'].replace({'Dealer':0,'Individual':1})
df.head()

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,3.35,5.59,27000,0,0,Manual,0,10
1,4.75,9.54,43000,1,0,Manual,0,11
2,7.25,9.85,6900,0,0,Manual,0,7
3,2.85,4.15,5200,0,0,Manual,0,13
4,4.60,6.87,42450,1,0,Manual,0,10


In [61]:
df['Transmission'] = df['Transmission'].replace({'Manual':0,'Automatic':1})
df.head()

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,3.35,5.59,27000,0,0,0,0,10
1,4.75,9.54,43000,1,0,0,0,11
2,7.25,9.85,6900,0,0,0,0,7
3,2.85,4.15,5200,0,0,0,0,13
4,4.60,6.87,42450,1,0,0,0,10


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 299 entries, 0 to 300
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Selling_Price   299 non-null    float64
 1   Present_Price   299 non-null    float64
 2   Kms_Driven      299 non-null    int64  
 3   Fuel_Type       299 non-null    int64  
 4   Seller_Type     299 non-null    int64  
 5   Transmission    299 non-null    int64  
 6   Owner           299 non-null    int64  
 7   age_of_the_car  299 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 21.0 KB


### 8. Separate the target and independent features.

In [64]:

X = df.drop(columns=['Selling_Price'], axis=1)
y = df['Selling_Price']

In [85]:
X.head()

,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,5.59,27000,0,0,0,0,10
1,9.54,43000,1,0,0,0,11
2,9.85,6900,0,0,0,0,7
3,4.15,5200,0,0,0,0,13
4,6.87,42450,1,0,0,0,10


In [86]:
y.head()

0    3.35
1    4.75
2    7.25
3    2.85
4    4.60
Name: Selling_Price, dtype: float64

### 9. Split the data into train and test.

In [72]:
## Split the data in to train and test 

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(209, 7) (90, 7)
(209,) (90,)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [76]:
rf = RandomForestRegressor()

In [77]:
rf.fit(X_train,y_train)

RandomForestRegressor()

In [83]:
## Accuracy check 

y_train_pred = rf.predict(X_train) 
y_test_pred = rf.predict(X_test)


accuracy_train = r2_score(y_train, y_train_pred)
accuracy_test = r2_score(y_test, y_test_pred)

print('Accuracy Train', accuracy_train)
print('Accuracy Test', accuracy_test)

Accuracy Train 0.9832421221915822
Accuracy Test 0.9015408482229719


### 11. Create a pickle file with an extension as .pkl

In [84]:
pickle.dump(rf, open('model.pkl','wb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

### b) Create app.py file and write the predict function

### 13. Deploy your app on Railway.app

In [ ]:
# Write the steps to be performed in order to deploy the application on railway.

### 14. Paste the URL of the heroku application below, and while submitting the solution submit the this notebook along with the source code.

### Happy Learning :)